In [215]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np

In [216]:
batch_size=32
learning_rate=1e-3
epochs=30

In [217]:
training_data = datasets.FashionMNIST(
    root="../Datasets",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="../Datasets",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # cha1 and 4 filters
        self.ConvLayers=nn.Sequential(
            nn.Conv2d(1,64,3),
            nn.Dropout(0.5),
            nn.MaxPool2d(3),
            nn.Conv2d(64,32,3),
            nn.Dropout(0.5),
            nn.MaxPool2d(2) 
        )
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(32 * 3 * 3, 10)
    def forward(self,x):
        x=self.ConvLayers(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

In [218]:
model=NeuralNetwork()
loss_fn=nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [219]:
for batch,(X,y) in enumerate(train_dataloader):
    print(model(X))
    break

tensor([[-0.3884,  0.0463, -0.0380, -0.1168, -0.0504,  0.1310,  0.3993, -0.1170,
          0.1477,  0.3052],
        [-0.3210, -0.0712,  0.3581,  0.0886, -0.2820,  0.2099,  0.3968, -0.0753,
         -0.1469,  0.4600],
        [-0.3025, -0.1474,  0.0893,  0.0085, -0.1952, -0.0548,  0.2527,  0.0852,
         -0.0315,  0.1013],
        [-0.3467,  0.0729,  0.1705, -0.0508, -0.1989,  0.0984,  0.2953,  0.1108,
         -0.0103,  0.1412],
        [-0.7932, -0.0979,  0.1578,  0.1915, -0.3488,  0.1880,  0.2216, -0.2032,
          0.1692,  0.2568],
        [-0.0941,  0.0575,  0.2720,  0.0532, -0.5110,  0.3188,  0.3148, -0.0984,
         -0.2096,  0.0330],
        [-0.3655,  0.1664, -0.0539, -0.0422, -0.0788, -0.0520,  0.4758, -0.2230,
          0.0671,  0.2231],
        [-0.0488, -0.1829,  0.2816,  0.0544, -0.2412,  0.5397,  0.1290, -0.1385,
          0.1338,  0.3293],
        [-0.1563,  0.1025,  0.3499,  0.0747, -0.2438,  0.1005,  0.1545,  0.0626,
         -0.0955,  0.0632],
        [-0.2416,  

In [226]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
def train(train_dataloader,test_dataloader, model, loss_fn, optimizer):
    size_train = len(train_dataloader.dataset)
    size_test = len(test_dataloader.dataset)
    correct_train=0
    train_loss=0
    correct_test=0
    test_loss=0
    num_batches_test = len(test_dataloader)
    num_batches_train = len(train_dataloader)
    for batch_no,(X,y) in enumerate(train_dataloader):
        X,y = X.to(device),y.to(device)
        pred = model(X)
        loss=loss_fn(pred,y)
        train_loss += loss_fn(pred, y).item()
        correct_train += (pred.argmax(1) == y).type(torch.float).sum().item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    with torch.no_grad():
        for X, y in test_dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct_test += (pred.argmax(1) == y).type(torch.float).sum().item()
    print(f"Train Error: Accuracy: {(correct_train/(size_train)):>0.3f} loss: {train_loss/num_batches_train:>7f} ")
    print(f"Test Error: Accuracy: {(100*(correct_test/size_test)):>0.3f}%, loss: {(test_loss/num_batches_test):>8f} \n")
            

In [227]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader,test_dataloader, model, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
Train Error: Accuracy: 0.883 loss: 0.319249 
Test Error: Accuracy: 87.570%, loss: 0.355972 

Epoch 2
-------------------------------
Train Error: Accuracy: 0.885 loss: 0.318302 
Test Error: Accuracy: 86.900%, loss: 0.368445 

Epoch 3
-------------------------------
Train Error: Accuracy: 0.884 loss: 0.316272 
Test Error: Accuracy: 87.310%, loss: 0.353643 

Epoch 4
-------------------------------
Train Error: Accuracy: 0.884 loss: 0.317357 
Test Error: Accuracy: 87.050%, loss: 0.358617 

Epoch 5
-------------------------------
Train Error: Accuracy: 0.884 loss: 0.315426 
Test Error: Accuracy: 86.840%, loss: 0.370116 

Epoch 6
-------------------------------
Train Error: Accuracy: 0.885 loss: 0.311697 
Test Error: Accuracy: 87.000%, loss: 0.369441 

Epoch 7
-------------------------------
Train Error: Accuracy: 0.885 loss: 0.313189 
Test Error: Accuracy: 87.230%, loss: 0.356453 

Epoch 8
-------------------------------
Train Error: Accuracy: 0.886 